# Stock Volatility and Google Trends

In this activity, you’ll analyze time series data about Apple to identify any correlations among Google Trends, the stock price returns, and the stock volatility.

Instructions

Do a time series analysis of Apple from October 2015 to October 2020. Evaluate if any predictable relationship exists among the Google Trends data, Apple’s stock price, and Apple’s volatility and return metrics. To do all of that, complete the steps in the following two sections.

#### Set Up and Visualize the Data

1. Read the time series data into two Pandas DataFrames, `df_stock` and `df_trends`. Set the “date” column as the index for each of these DataFrames.

2. Concatenate the `df_stock` DataFrame to the `df_trends` DataFrame, creating a single DataFrame named `df_apple`.

3. Create a plot of the concatenated DataFrame by using hvPlot to visually analyze any trends and correlations. Do you observe any seasonal patterns?

    > **Hint** You might find it useful to refer to the [Apple Events page](https://www.apple.com/apple-events/) and the [Timeline of Apple Inc. products](https://en.wikipedia.org/wiki/Timeline_of_Apple_Inc._products) to spotlight moments over time that could have triggered a particular pattern.

#### Build Time Trends

1. Note that on September 10, 2019, Apple organized an event where it presented the new iPhone 11 family, the Apple Watch Series 5, and a new iPad. This event gained worldwide attention.

2. Closely examine the data from March 1, 2019 to January 31, 2020.

3. Create a plot by using hvPlot, and then identify whether both time series indicate a common trend that might correspond to this narrative.

4. Before seeking any correlations between these time series, add columns to the `df_apple` DataFrame to analyze the impact of the Google Trends data on the weekly returns and stock volatility, as follows:

    1. Use the Pandas `shift` function to add a new column that lags the Google Trends data by one period.

        > **Hint** Google Trends data is reported every week on Sunday, so you have a weekly period in this time series.

    2. Use the Pandas `pct_change` function to add a new column that has the weekly price returns.

    3. Compute the rolling stock volatility for Apple by using the following Pandas method:
        `df_apple["weekly_volatility"] = df_apple["close"].pct_change().rolling(window=4).std()`

5. Use the Pandas `corr` function to compute the correlations among the lagged Google Trends data, the price returns, and the stock volatility. Does any predictable relationship exist?


References

- [Pandas concat function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

- [Pandas rolling function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html)

- [Pandas std function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.std.html)

- [Pandas corr function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)


In [38]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

## Set up and Visualize the Data

### Step 1: Read the time series data into two Pandas DataFrames, `df_stock` and `df_trends`. Set the “date” column as the index for each of these DataFrames.

In [39]:
# Read the data from the apple-price.csv file into a Pandas DataFrame
file_path_prices = Path("../Resources/aapl-price.csv")
file_path_trends = Path("../Resources/apple-trends.csv")

df_stock_prices = pd.read_csv(
    file_path_prices,
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True
)
df_stock_trends = pd.read_csv(
    file_path_trends,
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True
)

# Review the DataFrames
display(df_stock_prices.head())



,close
date,
2015-10-19,27.93
2015-10-20,28.44
2015-10-21,28.44
2015-10-22,28.88
2015-10-23,29.77


In [40]:
# Read the data from the apple-trends.csv file into a Pandas DataFrame
display(df_stock_trends.head())

,trend-worldwide
date,
2015-10-23,32
2015-10-30,33
2015-11-06,32
2015-11-13,32
2015-11-20,31


#### Step 2: Concatenate the `df_stock` DataFrame to the `df_trends` DataFrame, creating a single DataFrame named `df_apple`.

> **Rewind** Concatenation: Two DataFrames that have the same `datetime` index can be merged using `pd.concat`. The `axis=1` parameter means that we're bringing the columns of the two DataFrames together.

In [41]:
# Concatenate Apple's stock price and Google trends data
# Use a parameter of axis=1 to ensure the concatenation by columns
# Chain the dropna function to remove any rows of data that do not 
# contain information in both columns
df_apple = pd.concat([df_stock_prices, df_stock_trends], axis=1).dropna()

# Review the df_apple DataFrame
df_apple


,close,trend-worldwide
date,,
2015-10-23,29.77,32.0
2015-10-30,29.88,33.0
2015-11-06,30.27,32.0
2015-11-13,28.09,32.0
2015-11-20,29.83,31.0
...,...,...
2020-09-11,112.00,37.0
2020-09-18,106.84,52.0
2020-09-25,112.28,42.0


### Step 3: Create a plot of the concatenated DataFrame by using hvPlot to visually analyze any trends and correlations. Do you observe any seasonal patterns?

> **Hint** You might find it useful to refer to the [Apple Events page](https://www.apple.com/apple-events/) and the [Timeline of Apple Inc. products](https://en.wikipedia.org/wiki/Timeline_of_Apple_Inc._products) to spotlight moments over time that could have triggered a particular pattern.

In [42]:
# Use hvplot to visualize the time series data in the df_apple DataFrame
df_apple.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

## Building Time Trends

### Step 1: Note that on September 10, 2019, Apple organized an event where it presented the new iPhone 11 family, the Apple Watch Series 5, and a new iPad. This event gained worldwide attention.

### Step 2: Closely examine the data from March 1, 2019 to January 31, 2020.

In [43]:
# Using the df_apple DataFrame, use the loc function to select a
# range of data from March 1st, 2019 to January 31st, 2020
df_spotlight = df_apple.loc["2019-03-01" : "2020-01-31"]

# Review the df_spotlight DataFrame
display(df_spotlight.head())
display(df_spotlight.tail())


,close,trend-worldwide
date,,
2019-03-01,43.74,27.0
2019-03-08,43.23,27.0
2019-03-15,46.53,27.0
2019-03-22,47.76,30.0
2019-03-29,47.49,34.0


,close,trend-worldwide
date,,
2020-01-03,74.36,38.0
2020-01-10,77.58,34.0
2020-01-17,79.68,32.0
2020-01-24,79.58,32.0
2020-01-31,77.38,31.0


### Step 3: Create a plot by using hvPlot, and then identify whether both time series indicate a common trend that might correspond to this narrative.

In [44]:
# Use hvPlot to visualize the df_spotlight DataFrame
df_spotlight.hvplot()


:NdOverlay   [Variable]
   :Curve   [date]   (value)

### Step 4: Before seeking any correlations between these time series, add columns to the `df_apple` DataFrame to analyze the impact of the Google Trends data on the weekly returns and stock volatility, as follows:

1. Use the Pandas `shift` function to add a new column that lags the Google Trends data by one period.

    > **Hint** Google Trends data is reported every week on Sunday, so you have a weekly period in this time series.

2. Use the Pandas `pct_change` function to add a new column that has the weekly price returns.

3. Compute the rolling stock volatility for Apple by using the following Pandas method:

    `df_apple["weekly_volatility"] = df_apple["close"].pct_change().rolling(window=4).std()`


In [45]:
# Create a column which lags Google trends by one week
# Use the shift function, and move the data down by one row
df_apple["lagged_trends"] = df_apple["trend-worldwide"] .shift()
df_apple.head()


,close,trend-worldwide,lagged_trends
date,,,
2015-10-23,29.77,32.0,NaN
2015-10-30,29.88,33.0,32.0
2015-11-06,30.27,32.0,33.0
2015-11-13,28.09,32.0,32.0
2015-11-20,29.83,31.0,32.0


In [46]:
# Create a column that contains the Apple weekly return data
# Use the pct_change function to calculate the weekly return values
df_apple["weekly_returns"] = df_apple["close"].pct_change()
df_apple.head()


,close,trend-worldwide,lagged_trends,weekly_returns
date,,,,
2015-10-23,29.77,32.0,NaN,NaN
2015-10-30,29.88,33.0,32.0,0.003695
2015-11-06,30.27,32.0,33.0,0.013052
2015-11-13,28.09,32.0,32.0,-0.072019
2015-11-20,29.83,31.0,32.0,0.061944


In [49]:
# Create a column of Apple weekly rolling stock volatility 
# Chain the pct_function, the rolling function and a 4 period window, and the std function 
df_apple["weekly_volatility"] = df_apple["close"]
df_apple["weekly_volatility"] = df_apple["close"].rolling(window = 4).std().pct_change()

df_apple = df_apple.dropna()
df_apple.head()


,close,trend-worldwide,lagged_trends,weekly_returns,weekly_volatility
date,,,,,
2015-11-20,29.83,31.0,32.0,0.061944,0.006199
2015-11-27,29.45,35.0,31.0,-0.012739,-0.031027
2015-12-04,29.76,33.0,35.0,0.010526,-0.137700
2015-12-11,28.30,33.0,33.0,-0.049059,-0.126217
2015-12-18,26.51,33.0,33.0,-0.063251,1.073112


### Step 5: Use the Pandas `corr` function to compute the correlations among the lagged Google Trends data, the price returns, and the stock volatility. Does any predictable relationship exist?

In [50]:
# Use the corr function to compute the correlation between the lagged Google Trends data, price returns, and stock volatility
df_apple[['lagged_trends', 'weekly_returns', 'weekly_volatility']].corr()


,lagged_trends,weekly_returns,weekly_volatility
lagged_trends,1.000000,0.026231,-0.020159
weekly_returns,0.026231,1.000000,0.006663
weekly_volatility,-0.020159,0.006663,1.000000
